# PyRAT subject ingestion

## Setup

Using local config file (see [01_pipeline](./01_pipeline.ipynb))

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session
import datajoint as dj

[2023-01-16 19:00:06,346][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-01-16 19:00:06,421][INFO]: Connected tobiasr@172.26.128.53:3306


In [ ]:
db_prefix = dj.config['custom'].get('database.prefix')
print('database Prefix:', db_prefix)

Manual entry

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'tobiasr'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass(prompt='Database password:')
dj.config['custom']['pyrat_client_token'] = getpass.getpass(prompt="Pyrat client token:")
dj.config['custom']['pyrat_user_token'] = getpass.getpass(prompt="Pyrat user token:")
dj.conn()

## Initial check of tables

In [2]:
# subject.User.delete(); 
# subject.Protocol.delete()
# subject.Line.delete(); 
# subject.Subject.delete()
print('User', len(subject.User()))
print('Protocol', len(subject.Protocol()))
print('Line', len(subject.Line()))
print('Mutation', len(subject.Mutation()))
print('Subject', len(subject.Subject()))
print('SubjectGenotype', len(subject.SubjectGenotype()))

User 8
Protocol 3
Line 2
Mutation 2
Subject 2
SubjectGenotype 2


In [ ]:
subject.User()


## Automated ingestion

The function is designed to list all proposed insertions and ask for a confirmation before entered into the schema.

In [4]:
from adamacs.schemas import subject
from adamacs.ingest.pyrat import PyratIngestion


In [ ]:
# PyratIngestion().ingest_ani
# mal("ROS-1483")
PyratIngestion().ingest_animal("WEZ*")

This function also permits wildcards when querying [the API](https://pyrat.uniklinik-bonn.de/pyrat-test/api/v2/specification/ui#/Listing/get_animals).

In [5]:
PyratIngestion().ingest_animal("WEZ-8701")

Connected
Found no live entries for WEZ-8701. Testing sacrificed.
Gathering users...
Gathering protocols...
Gathering lines/mutations...
Gathering subjects...
--- PyRAT items to be inserted ---
User(s):  [] 

Protocol(s):  [] 

Line(s):  [] 

Mutation(s):  [] 

Subjects:  [] 



In [ ]:
PyratIngestion().ingest_animal("ROS-1438")

## Confirm entry

In [ ]:
print('User', len(subject.User()))
print('Protocol', len(subject.Protocol()))
print('Line', len(subject.Line()))
print('Mutation', len(subject.Mutation()))
print('Subject', len(subject.Subject()))
print('SubjectGenotype', len(subject.SubjectGenotype()))

In [6]:
subject.User()

user_id,name,shorthand TR: added name_FirstInitialSurname shorthand,initials Update after pyrat ingestion,email TR: for completeness' sake also email,"lab short lab name, pyrat labid"
1,Rose Tobias,tobiasr,TR,trose@uni-bonn.de,Rose
2,Kück Laura,laurak,LK,laura.kueck@ukbonn.de,Rose
3,Krasilshchikova Natalia,nataliak,NK,nkra1@uni-bonn.de,Rose
4,Bühler Daniel,danielb,DB,Db247@uni-bonn.de,Rose
5,Luxem Kevin,kevinl,KL,luxemk@uni-bonn.de,Rose
6,Jung Jisoo,jisooj,JJ,jjun1@uni-bonn.de,Rose
7,Narayanamurthy Rukhmani,rukhun,RN,rnar@uni-bonn.de,Rose
8,Kremers Leon,leonk,LE,leon.kremers@uni-bonn.de,Rose


In [7]:
p = subject.Subject()


In [8]:
print(p)

*subject     earmark     sex     birth_date     death_date     generation     parent_ids owner_id     responsible_id line     protocol      
+----------+ +---------+ +-----+ +------------+ +------------+ +------------+ +--------+ +----------+ +------------+ +------+ +------------+
ROS-1438     G152        M       2022-06-22T13: 2022-12-07T14:                =BLOB=     1            2              373      81-02.04.2019.
WEZ-8701     T453        M       2021-10-19T00: 2022-09-19T19:                =BLOB=     1            2              376      01_Organentnah
 (Total: 2)



In [ ]:
query = subject.Subject() & 'subject = "WEZ-8701"'
l = subject.Subject.fetch('death_date').tolist

In [ ]:
print(l())

In [ ]:
l

In [ ]:
l['subject']

In [ ]:
l()[0]
